## This code illustrates how to get S&P500 index constituents and their identifiers from CRSP and Compustat
- Researchers used to be able to extract index membership information from Compustat's "comp.idxcst_his" data. Now that S&P pulled this piece of data off from WRDS platform, we have to turn to CRSP to get S&P500 Index membership data. 
- Unfortunately, there is no easy way to uncover constituents info for the other indices covered by "comp.idxcst_his".

In [1]:
#############################################
# S&P 500 Index Constituents                #
# Qingyi (Freda) Song Drechsler             #
# Date: October 2020                        #
# Update: November 2024 for CRSP CIZ format #
############################################# 

import pandas as pd
import wrds

### Step 1: Connect to WRDS

In [2]:
###################
# Connect to WRDS #
###################
conn=wrds.Connection()

Loading library list...
Done


### Step 2: Get SP500 Index Membership from CRSP
- I opt for the monthly frequency of the data, but one can choose to work with crsp.dsp500list if more precise date range is needed.

In [12]:
sp500 = conn.raw_sql("""
                        select *
                        from crsp.msp500list_v2
                        where mbrstartdt <= '12/31/2024'
                        and mbrenddt >= '01/01/2020'
                        order by mbrstartdt;
                        """, date_cols=['mbrstartddt', 'mbrenddt'])

sp500.head()

,permno,indno,mbrstartdt,mbrenddt,mbrflg,indfam
0,15720,1000500,1925-12-31,2024-12-31,NORM,1100500
1,11404,1000500,1925-12-31,2024-12-31,NORM,1100500
2,11850,1000500,1925-12-31,2024-12-31,NORM,1100500
3,10145,1000500,1925-12-31,2024-12-31,NORM,1100500
4,14541,1000500,1925-12-31,2024-12-31,NORM,1100500


In [13]:
sp500_full = sp500
sp500_full.head(5)

,permno,indno,mbrstartdt,mbrenddt,mbrflg,indfam
0,15720,1000500,1925-12-31,2024-12-31,NORM,1100500
1,11404,1000500,1925-12-31,2024-12-31,NORM,1100500
2,11850,1000500,1925-12-31,2024-12-31,NORM,1100500
3,10145,1000500,1925-12-31,2024-12-31,NORM,1100500
4,14541,1000500,1925-12-31,2024-12-31,NORM,1100500


### Step 4: Add Compustat Identifiers
- Link with Compustat's GVKEY and IID if need to work with fundamental data
- Linkage is done through crsp.ccmxpf_linktable

In [17]:
# Linking with Compustat through CCM

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm = ccm.loc[(ccm['linkdt'] <= '12/31/2024')\
              &(ccm['linkenddt'] >= '01/01/2020')]

In [25]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['mbrenddt']>=sp500ccm['linkdt'])\
                        &(sp500ccm['mbrstartdt']<=sp500ccm['linkenddt'])]

sp500ccm['mbrstartdt'] = pd.to_datetime(sp500ccm['mbrstartdt'])
sp500ccm['mbrenddt'] = pd.to_datetime(sp500ccm['mbrenddt'])
sp500ccm.sample(5)

,permno,indno,mbrstartdt,mbrenddt,mbrflg,indfam,gvkey,iid,linktype,linkprim,linkdt,linkenddt
331,79057,1000500,2010-07-15,2024-12-31,NORM,1100500,028034,01,LC,P,1993-03-25,2025-04-11 18:07:12.380618
179,46886,1000500,1997-09-15,2024-12-31,NORM,1100500,006304,01,LU,P,1980-10-08,2025-04-11 18:07:12.380618
569,23876,1000500,2023-08-25,2024-12-31,NORM,1100500,041897,01,LC,P,2023-05-04,2025-04-11 18:07:12.380618
290,88352,1000500,2008-06-02,2024-12-31,NORM,1100500,136725,01,LC,P,2000-06-13,2025-04-11 18:07:12.380618
76,59459,1000500,1976-07-01,2024-12-31,NORM,1100500,062689,05,LC,P,2004-04-02,2025-04-11 18:07:12.380618


In [26]:
# Define Quarter End Dates
from pandas.tseries.offsets import QuarterEnd
quarters = pd.date_range(start='2020-01-01', end='2024-12-31', freq='QE')

quarters

DatetimeIndex(['2020-03-31', '2020-06-30', '2020-09-30', '2020-12-31',
               '2021-03-31', '2021-06-30', '2021-09-30', '2021-12-31',
               '2022-03-31', '2022-06-30', '2022-09-30', '2022-12-31',
               '2023-03-31', '2023-06-30', '2023-09-30', '2023-12-31',
               '2024-03-31', '2024-06-30', '2024-09-30', '2024-12-31'],
              dtype='datetime64[ns]', freq='QE-DEC')

In [27]:
# For Each Quarter, Filter Active Membership
quarter_dict = {}

for date in quarters:
    quarter_label = f"{date.year}Q{date.quarter}"
    active = sp500ccm[
        (sp500ccm['mbrstartdt'] <= date) &
        (sp500ccm['mbrenddt'] >= date)
    ]
    active = active.drop_duplicates(subset='gvkey')  # keep one row per permno
    active = active[['gvkey', 'permno']].dropna()
    quarter_dict[quarter_label] = active

for quarter, df in quarter_dict.items():
    print(f"{quarter}: {len(df)} companies")

2020Q1: 501 companies
2020Q2: 501 companies
2020Q3: 501 companies
2020Q4: 501 companies
2021Q1: 501 companies
2021Q2: 501 companies
2021Q3: 501 companies
2021Q4: 501 companies
2022Q1: 501 companies
2022Q2: 501 companies
2022Q3: 501 companies
2022Q4: 501 companies
2023Q1: 501 companies
2023Q2: 501 companies
2023Q3: 501 companies
2023Q4: 501 companies
2024Q1: 501 companies
2024Q2: 501 companies
2024Q3: 502 companies
2024Q4: 501 companies


In [ ]:
quarter_dict

In [29]:
# 1. Get GVKEY–CIK mapping from Compustat
gvkey_cik = conn.raw_sql("""
    SELECT gvkey, conml, cik
    FROM comp.company
    WHERE cik IS NOT NULL
""")

# Check result
gvkey_cik = gvkey_cik.drop_duplicates(subset='gvkey').dropna()
gvkey_cik['cik'] = gvkey_cik['cik'].astype(str)
gvkey_cik.head()

,gvkey,conml,cik
0,001001,A & M Food Services Inc,0000723576
1,001002,AAI Corp,0001306124
2,001003,A.A. Importing Co Inc,0000730052
3,001004,AAR Corp,0000001750
4,001007,ABKCO Industries Inc,0000001882


In [31]:
for quarter in quarter_dict:
    gvkeys = quarter_dict[quarter]
    quarter_ciks = pd.merge(gvkeys, gvkey_cik, on='gvkey', how='inner')[['gvkey', 'cik']].drop_duplicates()
    quarter_dict[quarter] = quarter_ciks

quarter_dict

{'2020Q1':       gvkey         cik
 0    009846  0000827052
 1    003413  0001047862
 2    004503  0000034088
 3    001300  0000773840
 4    002991  0000093410
 ..      ...         ...
 496  025313  0001757898
 497  165746  0001335258
 498  024405  0000877212
 499  020116  0001590955
 500  030098  0001699150
 
 [501 rows x 2 columns],
 '2020Q2':       gvkey         cik
 0    009846  0000827052
 1    003413  0001047862
 2    004503  0000034088
 3    001300  0000773840
 4    002991  0000093410
 ..      ...         ...
 496  162887  0001093557
 497  011376  0000105770
 498  010789  0000860731
 499  002220  0000012208
 500  126721  0001094285
 
 [501 rows x 2 columns],
 '2020Q3':       gvkey         cik
 0    009846  0000827052
 1    003413  0001047862
 2    004503  0000034088
 3    001300  0000773840
 4    002991  0000093410
 ..      ...         ...
 496  002220  0000012208
 497  126721  0001094285
 498  023238  0001370637
 499  020228  0001596783
 500  010453  0000097210
 
 [501 rows x 2

In [34]:
# Merge all quarters into one DataFrame with a "quarter" column
all_quarters_df = []

for quarter, df in quarter_dict.items():
    df['quarter'] = quarter
    all_quarters_df.append(df)

final_cik_df = pd.concat(all_quarters_df, ignore_index=True)
final_cik_df.to_csv("sp500_all_quarters_cik.csv", index=False)

In [37]:
# Load all S&P 500 CIKs by quarter
cik_df = pd.read_csv("sp500_all_quarters_cik.csv")
cik_df['quarter'] = cik_df['quarter'].astype(str)
cik_df['cik'] = cik_df['cik'].astype(int)

In [38]:
cik_df

,gvkey,cik,quarter
0,9846,827052,2020Q1
1,3413,1047862,2020Q1
2,4503,34088,2020Q1
3,1300,773840,2020Q1
4,2991,93410,2020Q1
...,...,...,...
10016,61380,922621,2024Q4
10017,10504,1811074,2024Q4
10018,184254,1858681,2024Q4
10019,122380,1069202,2024Q4
